**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import random

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten

import skvideo
path = r"C:\Users\Samue\Anaconda3\Lib\site-packages\skvideo\io"
path2 = r"C:\Users\Samue\Desktop\ffmpeg\ffmpeg-20200115-0dc0837-win64-static\bin"
skvideo.setFFmpegPath(path)

Using TensorFlow backend.
C:\Users\Samue\Anaconda3\lib\site-packages\skvideo\__init__.py:306: UserWarning: ffmpeg/ffprobe not found in path: C:\Users\Samue\Anaconda3\Lib\site-packages\skvideo\io
  warnings.warn("ffmpeg/ffprobe not found in path: " + str(path), UserWarning)


In [2]:
skvideo.setFFmpegPath(path2)

C:\Users\Samue\Anaconda3\lib\site-packages\skvideo\__init__.py:306: UserWarning: ffmpeg/ffprobe not found in path: C:\Users\Samue\Desktop\ffmpeg\ffmpeg-20200115-0dc0837-win64-static\bin
  warnings.warn("ffmpeg/ffprobe not found in path: " + str(path), UserWarning)


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

__Answer 1__

$\epsilon$  allow to continue to explore the other action in order to have more information on there performances and choose the best action in the future. It is essential in the  Exploration-Exploitation to continue to explore.
In order to balance the random choices made, the player also chooses the actual best action to its knowlegde with probability 1- $\epsilon$

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

__Answer 2__

self.board is the board on which one plays, it is where we store the available rewards and when the cat is on a cell, it gets the reward : reward = self.board[self.x, self.y]

Self.position gives us the position of the cat in the board with value 1, and with value -1 the case where he can't go.


## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return int(np.random.randint(0,4,1))

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            #loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 15.5/19.0. Average score (-3.5)
Win/lose count 6.0/11.0. Average score (-4.25)
Win/lose count 4.0/10.0. Average score (-4.833333333333333)
Win/lose count 17.5/24.0. Average score (-5.25)
Win/lose count 12.0/12.0. Average score (-4.2)
Win/lose count 9.0/19.0. Average score (-5.166666666666667)
Win/lose count 7.5/7.0. Average score (-4.357142857142857)
Win/lose count 6.0/11.0. Average score (-4.4375)
Win/lose count 7.5/17.0. Average score (-5.0)
Win/lose count 4.5/14.0. Average score (-5.45)
Final score: -5.45


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer 5__

We have by definition :

\begin{equation}
    \begin{aligned}
        Q^\pi(s,a)=E_{p^{\pi}} (\sum_{t=0}^{t= T} \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a) \>  \newline
        Q^\pi(s,a)=E_{p^{\pi}} ( \gamma^{0}r(s_{0},a_{0}) + \sum_{t=1}^{t=T} \gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}= a) \> \newline
        Q^\pi(s,a)=E_{p^{\pi}} ( \gamma^{0}r(s_{0},a_{0}) + \gamma*\sum_{t=0}^{t=T-1} \gamma^{t}r(s_{t+1},a_{t+1})|s_{0}=s,a_{0}= a) \>
    \end{aligned}
\end{equation}
Thus, we have : 

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, the optimal Q function is:
\begin{equation}
    \begin{aligned}
    Q^*(s,a)&=\max_{\pi}Q^\pi(s,a) \> . \newline
    Q^*(s,a)&= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
    \end{aligned}
\end{equation}

As $\pi$ is the strategy, we have :

\begin{equation}
\max_{\pi} E_{(s',a')\sim p(.|s,a)} = E_{s'\sim \pi^*(.|s,a)}max_{a'}
\end{equation}

finally, as $r(s,a)$ is independant of a', we have : 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}

In order for $\mathcal{L}(\theta)$ to be a plausible objective, it needs to be positive (as the expectency of a positive variable) and to converge to zero when $Q^{\pi}(s,a) \rightarrow Q^{*}(s,a)$. Which is indeed the case because :

\begin{equation}
    \begin{aligned}
        &Q^{\pi}(s,a) \rightarrow Q^{*}(s,a) \newline
        \iff &E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] - E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \rightarrow 0 \newline
        \iff &E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')-Q^{\pi}(s',a')]  \rightarrow 0 \newline
        \iff &\mathcal{L}(\theta) \rightarrow 0
    \end{aligned}
\end{equation}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        actual_length = len(self.memory)
        
        if(actual_length>self.max_memory):
            self.memory.pop(0)
        self.memory.append(m)

    def random_access(self):
        i = np.random.randint(0, len(self.memory), size=1)[0]
        return self.memory[i]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## pick a random itteration
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()
            
            # predict target q
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
                
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
                
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(30,activation ='relu')) #simple one hidden layer network (see performance of other methods below)
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Epoch 000/010 | Loss 0.0077 | Win/lose count 0.5/4.0 (-3.5)
Epoch 001/010 | Loss 0.0109 | Win/lose count 2.5/1.0 (1.5)
Epoch 002/010 | Loss 0.0341 | Win/lose count 2.5/3.0 (-0.5)
Epoch 003/010 | Loss 0.0568 | Win/lose count 2.0/0 (2.0)
Epoch 004/010 | Loss 0.0271 | Win/lose count 2.0/6.0 (-4.0)
Epoch 005/010 | Loss 0.0078 | Win/lose count 5.0/3.0 (2.0)
Epoch 006/010 | Loss 0.0072 | Win/lose count 2.0/4.0 (-2.0)
Epoch 007/010 | Loss 0.0170 | Win/lose count 3.0/4.0 (-1.0)
Epoch 008/010 | Loss 0.0142 | Win/lose count 1.0/4.0 (-3.0)
Epoch 009/010 | Loss 0.0102 | Win/lose count 1.5/2.0 (-0.5)


***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

__Answer__
I tried differents models : changing the number of convolution Layer, the activation function, the size of the kernel.

My best results are for : 2 hidden Convolution Layer ((50,(2,2) and then (30,(2,2)) with the relu activation

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        
        model.add(Conv2D(30,(2,2),activation='relu'))
        
        model.add(Flatten())
        
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        
        self.model = model

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train0.mp4'))

Epoch 000/010 | Loss 0.0185 | Win/lose count 4.5/3.0 (1.5)
Epoch 001/010 | Loss 0.0049 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/010 | Loss 0.0031 | Win/lose count 3.5/15.0 (-11.5)
Epoch 003/010 | Loss 0.0029 | Win/lose count 1.5/1.0 (0.5)
Epoch 004/010 | Loss 0.0220 | Win/lose count 5.0/5.0 (0.0)
Epoch 005/010 | Loss 0.0525 | Win/lose count 5.0/1.0 (4.0)
Epoch 006/010 | Loss 0.0623 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/010 | Loss 0.0488 | Win/lose count 1.5/3.0 (-1.5)
Epoch 008/010 | Loss 0.0015 | Win/lose count 4.5/2.0 (2.5)
Epoch 009/010 | Loss 0.0021 | Win/lose count 7.0/1.0 (6.0)


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.1)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/0. Average score (1.0)
Win/lose count 0.5/1.0. Average score (0.25)
Win/lose count 1.5/2.0. Average score (0.0)
Win/lose count 0.5/3.0. Average score (-0.625)
Win/lose count 1.0/2.0. Average score (-0.7)
Win/lose count 1.5/1.0. Average score (-0.5)
Win/lose count 1.0/0. Average score (-0.2857142857142857)
Win/lose count 1.0/1.0. Average score (-0.25)
Win/lose count 1.0/3.0. Average score (-0.4444444444444444)
Win/lose count 2.5/1.0. Average score (-0.25)
Final score: -0.25
Test of the FC
Win/lose count 0/3.0. Average score (-3.0)
Win/lose count 0.5/2.0. Average score (-2.25)
Win/lose count 0.5/2.0. Average score (-2.0)
Win/lose count 2.0/6.0. Average score (-2.5)
Win/lose count 0.5/0. Average score (-1.9)
Win/lose count 1.5/0. Average score (-1.3333333333333333)
Win/lose count 1.0/2.0. Average score (-1.2857142857142858)
Win/lose count 1.5/5.0. Average score (-1.5625)
Win/lose count 0.5/3.0. Average score (-1.6666666666666667)
Win/lose count 1.5/1.0. 

In [17]:
HTML(display_videos('cnn_test0.mp4'))

In [18]:
HTML(display_videos('fc_test0.mp4'))

__Answer__

The main issue that we can observe is that the algorithms tends to not explore the map, the mousse stay arround it's initial position. It makes small circle and take the nearest ressource if tere is ressources.

This method will not decrease the score but will not help to improve it. The mouse is too carefull.



***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [19]:
def train_explore(agent,env,epoch,decay_parameter_epsilon=0.3,prefix=''):
    # We use the template of the previous train function
    # We compute a new training procedure in order to increase the exploration 
    # Decay_parameter_epsilon in order to use the decreasing epsilon-greedy exploration
    
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        
        state = env.reset()
        game_over = False

        win = 0
        lose = 0
        
        ## the following line is the important line here
        agent.set_epsilon(agent.epsilon*(1-decay_parameter_epsilon))
        
        while not game_over:
            action = agent.act(state)

            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        if e % 10 == 0:
            env.draw(prefix+str(e))

        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

# We now use as template the class Environment
## In Environment exploring we will have to change n_state to 3 because you will use one more layer

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)) #define maluses when going to a previously visited position
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        #During the training phase going back to a position where the rat have already been before tends to decrease the 
        #total reward hence there is this train parameter that we have added (it tries to enforce the exploration)

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1     
        
        
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        #At the begining the malus_position array must be setted to zero
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state


In [20]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore0.mp4'))

Epoch 000/010 | Loss 0.0022 | Win/lose count 3.5/21.100000000000033 (-17.600000000000033)
Epoch 001/010 | Loss 0.0025 | Win/lose count 2.0/19.900000000000016 (-17.900000000000016)
Epoch 002/010 | Loss 0.0050 | Win/lose count 10.0/19.69999999999999 (-9.699999999999989)
Epoch 003/010 | Loss 0.0285 | Win/lose count 6.5/25.100000000000055 (-18.600000000000055)
Epoch 004/010 | Loss 0.0105 | Win/lose count 4.0/19.000000000000004 (-15.000000000000004)
Epoch 005/010 | Loss 0.0073 | Win/lose count 10.0/18.5 (-8.5)
Epoch 006/010 | Loss 0.0027 | Win/lose count 12.5/16.199999999999964 (-3.6999999999999638)
Epoch 007/010 | Loss 0.0034 | Win/lose count 11.5/17.899999999999977 (-6.399999999999977)
Epoch 008/010 | Loss 0.0061 | Win/lose count 2.0/19.0 (-17.0)
Epoch 009/010 | Loss 0.0033 | Win/lose count 5.5/18.8 (-13.3)


In [21]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 5.0/0. Average score (5.0)
Win/lose count 2.0/0. Average score (3.5)
Win/lose count 7.0/1.0. Average score (4.333333333333333)
Win/lose count 14.5/0. Average score (6.875)
Win/lose count 4.0/2.0. Average score (5.9)
Win/lose count 1.0/0. Average score (5.083333333333333)
Win/lose count 6.5/1.0. Average score (5.142857142857143)
Win/lose count 6.5/0. Average score (5.3125)
Win/lose count 2.5/0. Average score (5.0)
Win/lose count 6.5/1.0. Average score (5.05)
Final score: 5.05


__Observation__

In regards of the results of this last method, we can see the outcome has been widely improved using the exploration techniques.

Visually, we see that the mouse tends to explore more the grid.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [22]:
class DQN_Mim(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        ''''same as for DQN Class'''
        super(DQN_Mim, self).__init__(epsilon = epsilon)

        self.discount = 0.99        
        self.grid_size = grid_size
        self.n_state = n_state
        self.memory = Memory(memory_size)
        self.batch_size = batch_size

    def learned_act(self, s):
        ''''same as for DQN Class'''
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])

    def reinforce(self, s_, a_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_[:,:,1:], a_]) 
        #In this case, we just need the current state and the action that has been applied
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, a_ = self.memory.random_access()
            input_states[i] = s_
            # In order to mimick the expert, we maximized the action made by the expert
            target_q[i,a_] = 10  
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        ''''same as for DQN Class'''
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        ''''same as for DQN Class'''
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_CNN_Mim(DQN_Mim):
    ''''CNN model with the best results over a test on differents hyperparamaters : activation function, number of Conv2D Layer, decay'''
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN_Mim, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [23]:
def train_mimick(expert,agent,env,epoch,prefix=''):
    ''''same as for DQN'''
    score = 0
    loss = 0    
    
    for e in range(epoch):
        state = env.reset()
        game_over = False

        win = 0
        lose = 0
        while not game_over:
            action = expert.act(state,train=False)
            prev_state = state
            state, reward, game_over = env.act(action)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state,  action)

        if e % 10 == 0:
            env.draw(prefix+str(e))

        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')


In [24]:
epochs_train = 50
expert = agent
agent_n = DQN_CNN_Mim(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=2)
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
train_mimick(expert,agent_n, env, epochs_train, prefix='cnn_train_mimick')


Epoch 000/050 | Loss 1.1419 | Win/lose count 8.5/1.0 (7.5)
Epoch 001/050 | Loss 0.3002 | Win/lose count 3.0/0 (3.0)
Epoch 002/050 | Loss 2.6543 | Win/lose count 8.0/1.0 (7.0)
Epoch 003/050 | Loss 0.9836 | Win/lose count 5.0/1.0 (4.0)
Epoch 004/050 | Loss 1.7042 | Win/lose count 9.0/0 (9.0)
Epoch 005/050 | Loss 0.6979 | Win/lose count 7.0/0 (7.0)
Epoch 006/050 | Loss 1.2850 | Win/lose count 4.5/0 (4.5)
Epoch 007/050 | Loss 0.7460 | Win/lose count 1.0/1.0 (0.0)
Epoch 008/050 | Loss 1.0415 | Win/lose count 6.0/0 (6.0)
Epoch 009/050 | Loss 1.1391 | Win/lose count 1.0/0 (1.0)
Epoch 010/050 | Loss 0.8140 | Win/lose count 4.5/0 (4.5)
Epoch 011/050 | Loss 1.0107 | Win/lose count 5.0/0 (5.0)
Epoch 012/050 | Loss 0.9314 | Win/lose count 3.5/0 (3.5)
Epoch 013/050 | Loss 0.4427 | Win/lose count 19.5/1.0 (18.5)
Epoch 014/050 | Loss 0.2982 | Win/lose count 7.5/0 (7.5)
Epoch 015/050 | Loss 0.5104 | Win/lose count 6.5/0 (6.5)
Epoch 016/050 | Loss 1.2354 | Win/lose count 5.5/0 (5.5)
Epoch 017/050 | Los

In [25]:
env_test = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_n,env_test,epochs_test,prefix='cnn_test_mimick')

Win/lose count 8.0/6.0. Average score (2.0)
Win/lose count 14.5/5.0. Average score (5.75)
Win/lose count 9.0/4.0. Average score (5.5)
Win/lose count 14.0/3.0. Average score (6.875)
Win/lose count 6.0/3.0. Average score (6.1)
Win/lose count 12.5/2.0. Average score (6.833333333333333)
Win/lose count 13.0/3.0. Average score (7.285714285714286)
Win/lose count 15.0/5.0. Average score (7.625)
Win/lose count 6.0/4.0. Average score (7.0)
Win/lose count 3.5/6.0. Average score (6.05)
Final score: 6.05


In [26]:
HTML(display_videos('cnn_test_mimick0.mp4'))

__Observation__

As we can see our algorithm that tries to mimick the expert performs better than the algorithm without exploration!

The final score for the copy is lower than the score obtained using the approach with exploration. 

One way to explain this difference is that it doesn't really learn to improve the score as it just tries to copy the original algorithm.
It could at best performs such as the expert algorithm but it couldn't exceed it (in average). 

In order to improve this algorithm, we define an approach where we would sometimes copy the expert and some others train the algorithm with new experiences.

***